In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Pandas'ın çok fazla satır/sütun göstermesi için ayar (opsiyonel)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

# --- 1. VERİ YÜKLEME VE TEMİZLEME ---

print("Adım 1: Veri Yükleniyor ve Temizleniyor...")
file_name = "Heart_Disease_Mortality_Data_Among_US_Adults__35___by_State_Territory_and_County___2019-2021.csv"

try:
    df = pd.read_csv(file_name)
    
    original_count = len(df)
    
    # ÖNEMLİ TEMİZLEME ADIMI:
    # 'Data_Value' (ölüm oranı) sütununda değeri olmayan (NaN) satırları kaldır.
    # Bu satırlar 'Insufficient Data' (Yetersiz Veri) notu olan satırlardır.
    df_clean = df.dropna(subset=['Data_Value'])
    
    clean_count = len(df_clean)
    print(f"Veri yüklendi. {original_count} satırdan {clean_count} kullanılabilir satır kaldı.")
    print("-" * 30)

except Exception as e:
    print(f"Dosya okuma hatası: {e}")
    # Kodun geri kalanının çalışmasını engellemek için burada durabiliriz
    exit()

Adım 1: Veri Yükleniyor ve Temizleniyor...
Veri yüklendi. 78792 satırdan 34430 kullanılabilir satır kaldı.
------------------------------


In [2]:
# --- 2. ANALİZ 1: EYALET BAZINDA GENEL ÖLÜM ORANLARI ---
# Projenin ilk sorusu: "Eyaletler arasındaki farklar nelerdir?"

print("\nAdım 2: Analiz 1 - Eyalet Bazında 'Genel' Ölüm Oranları")

# Veriyi filtrele:
# - Sadece 'Eyalet' (State) seviyesi
# - Sadece 'Genel' (Overall) Cinsiyet
# - Sadece 'Genel' (Overall) Irk/Etnisite
df_state_overall = df_clean[
    (df_clean['GeographicLevel'] == 'State') &
    (df_clean['Stratification1'] == 'Overall') &
    (df_clean['Stratification2'] == 'Overall')
]

# Eyaletlere göre grupla (birden fazla yıl varsa ortalamasını al) ve ölüm oranına göre sırala
state_mortality_overall = df_state_overall.groupby('LocationDesc')['Data_Value'].mean().sort_values(ascending=False)

print("En Yüksek Ölüm Oranına Sahip 10 Eyalet (Genel):")
print(state_mortality_overall.head(10))

print("\nEn Düşük Ölüm Oranına Sahip 10 Eyalet (Genel):")
print(state_mortality_overall.tail(10))
print("-" * 30)


Adım 2: Analiz 1 - Eyalet Bazında 'Genel' Ölüm Oranları
En Yüksek Ölüm Oranına Sahip 10 Eyalet (Genel):
LocationDesc
Guam             571.2
Oklahoma         477.7
Mississippi      467.8
Alabama          451.9
Arkansas         438.2
Louisiana        427.8
Tennessee        411.2
West Virginia    398.4
Kentucky         398.1
Nevada           393.1
Name: Data_Value, dtype: float64

En Düşük Ölüm Oranına Sahip 10 Eyalet (Genel):
LocationDesc
Alaska                        272.2
Connecticut                   270.6
Washington                    269.8
Oregon                        267.3
Colorado                      252.6
Massachusetts                 250.8
Hawaii                        238.7
Minnesota                     231.4
Puerto Rico                   199.1
Virgin Islands of the U.S.    181.1
Name: Data_Value, dtype: float64
------------------------------


In [3]:

# --- 3. ANALİZ 2: DEMOGRAFİK KIRILIM (CİNSİYET) ---
# Projenin ikinci sorusu: "Bu farklılıklar cinsiyetle ne ölçüde ilişkilidir?"

print("\nAdım 3: Analiz 2 - Eyalet Bazında 'Cinsiyet' Karşılaştırması")

# Veriyi filtrele:
# - Sadece 'Eyalet' (State) seviyesi
# - Cinsiyet 'Male' veya 'Female' (Bu kez 'Overall' değil)
# - Irk/Etnisite 'Overall' (Irk etkisini izole etmek için)
df_state_gender = df_clean[
    (df_clean['GeographicLevel'] == 'State') &
    (df_clean['Stratification1'].isin(['Male', 'Female'])) &
    (df_clean['Stratification2'] == 'Overall')
]

# Veriyi pivotla: Her eyalet için 'Male' ve 'Female' değerlerini sütunlara getir
state_gender_pivot = df_state_gender.groupby(
    ['LocationDesc', 'Stratification1']
)['Data_Value'].mean().unstack()

# Erkek ve Kadın oranları arasındaki farkı hesapla
state_gender_pivot['Fark (Erkek - Kadın)'] = state_gender_pivot['Male'] - state_gender_pivot['Female']

# Farkın en yüksek olduğu eyaletleri sırala
state_gender_pivot = state_gender_pivot.sort_values(by='Fark (Erkek - Kadın)', ascending=False)

print("Erkek ve Kadın Arasındaki Ölüm Oranı Farkının En Yüksek Olduğu 10 Eyalet:")
print(state_gender_pivot.head(10))
print("-" * 30)



Adım 3: Analiz 2 - Eyalet Bazında 'Cinsiyet' Karşılaştırması
Erkek ve Kadın Arasındaki Ölüm Oranı Farkının En Yüksek Olduğu 10 Eyalet:
Stratification1           Female   Male  Fark (Erkek - Kadın)
LocationDesc                                                 
Guam                       444.6  719.4                 274.8
Mississippi                364.4  596.8                 232.4
Oklahoma                   376.9  596.7                 219.8
Alabama                    356.5  570.3                 213.8
Louisiana                  333.0  544.9                 211.9
Arkansas                   344.8  549.1                 204.3
Tennessee                  319.2  523.3                 204.1
Nevada                     298.6  496.6                 198.0
Kentucky                   309.5  505.9                 196.4
Northern Mariana Islands   204.2  399.7                 195.5
------------------------------


In [4]:

# --- 4. ANALİZ 3: MEKÂNSAL ANALİZ (İLÇE BAZINDA HOTSPOT) ---
# Projenin bir diğer sorusu: "İlçeler arasındaki farklar nelerdir?" (Riskli bölgeler)

print("\nAdım 4: Analiz 3 - İlçe (County) Bazında En Yüksek Riskli Bölgeler (Hotspots)")

# Veriyi filtrele:
# - Sadece 'İlçe' (County) seviyesi
# - Sadece 'Genel' (Overall) Cinsiyet
# - Sadece 'Genel' (Overall) Irk/Etnisite
df_county_overall = df_clean[
    (df_clean['GeographicLevel'] == 'County') &
    (df_clean['Stratification1'] == 'Overall') &
    (df_clean['Stratification2'] == 'Overall')
]

# İlçeleri eyaletleriyle birlikte grupla (çünkü "Park County" birçok eyalette olabilir)
# 'LocationAbbr' (Eyalet Kısaltması) ve 'LocationDesc' (İlçe Adı) kullan
county_mortality = df_county_overall.groupby(
    ['LocationAbbr', 'LocationDesc']
)['Data_Value'].mean().sort_values(ascending=False)

print("En Yüksek Ölüm Oranına Sahip İlk 20 İlçe (Genel):")
print(county_mortality.head(20))
print("-" * 30)



Adım 4: Analiz 3 - İlçe (County) Bazında En Yüksek Riskli Bölgeler (Hotspots)
En Yüksek Ölüm Oranına Sahip İlk 20 İlçe (Genel):
LocationAbbr  LocationDesc     
LA            Franklin Parish      859.0
KY            Perry County         835.6
AL            Walker County        834.0
LA            Caldwell Parish      832.4
MO            Pemiscot County      770.6
KY            Wolfe County         761.6
GA            Clinch County        757.0
LA            Tensas Parish        750.4
KY            Breathitt County     738.8
TN            Lake County          727.0
SC            Marlboro County      725.0
MS            Tunica County        724.1
              Yalobusha County     720.6
VA            Martinsville City    713.2
MS            Leflore County       701.5
KY            Fulton County        698.6
OK            Muskogee County      697.6
SD            Buffalo County       688.8
TN            Carroll County       681.0
              Cocke County         680.3
Name: Data_Value, d

In [ ]:
# --- 5. GÖRSELLEŞTİRME (Analiz 1 için örnek) ---
# Analiz 1'deki (Eyalet Bazında Genel) sonuçların grafiğini çizelim

print("\nAdım 5: Görselleştirme - Eyalet Bazlı Bar Grafiği (Analiz 1)")

plt.figure(figsize=(12, 16)) # Grafik boyutunu ayarla
sns.barplot(
    x=state_mortality_overall.values, 
    y=state_mortality_overall.index,
    palette="coolwarm_r" # Yüksekler kırmızı, düşükler mavi
)
plt.title('ABD Eyaletlerine Göre Genel Kalp Hastalığı Ölüm Oranları (100.000 Kişi Başına)', fontsize=16)
plt.xlabel('Yaşa Göre Düzeltilmiş Ölüm Oranı', fontsize=12)
plt.ylabel('Eyalet / Bölge', fontsize=12)
plt.tight_layout() # Grafiğin sığmasını sağla

# Grafiği dosyaya kaydet
output_image_file = "eyalet_olum_oranlari_genel.png"
plt.savefig(output_image_file, dpi=300)

print(f"Grafik '{output_image_file}' olarak kaydedildi.")
# Eğer Jupyter Notebook kullanmıyorsanız, grafiği görmek için plt.show() ekleyebilirsiniz.
# plt.show()

print("\nBaşlangıç betiği tamamlandı. Artık bu kodları geliştirerek kendi analizlerinizi yapabilirsiniz.")
print("Örneğin: 'Analiz 2' ve 'Analiz 3' için de grafikler çizebilir veya 'Analiz 2'yi Irk/Etnisite ('Stratification2') için yapabilirsiniz.")

In [ ]:
# --- 2. YENİ ANALİZ: EYALET BAZINDA CİNSİYET FARKI (Erkek vs Kadın) ---
print("\nAdım 2: Analiz - Eyalet Bazında 'Cinsiyet' Farklılıkları")

# Veriyi filtrele:
# - Sadece 'Eyalet' (State) seviyesi
# - Cinsiyet 'Male' veya 'Female' (Bu kez 'Overall' değil)
# - Irk/Etnisite 'Overall' (Irk etkisini izole etmek için)
df_state_gender = df_clean[
    (df_clean['GeographicLevel'] == 'State') &
    (df_clean['Stratification1'].isin(['Male', 'Female'])) &
    (df_clean['Stratification2'] == 'Overall')
]

# Veriyi pivotla: Her eyalet için 'Male' ve 'Female' değerlerini sütunlara getir
# (Birden fazla yıl varsa ortalamasını al)
state_gender_pivot = df_state_gender.groupby(
    ['LocationDesc', 'Stratification1']
)['Data_Value'].mean().unstack()

# 'Male' veya 'Female' verisi eksik olan eyaletleri (örn. bazı bölgeler) temizle
state_gender_pivot = state_gender_pivot.dropna(subset=['Male', 'Female'])

# Erkek ve Kadın oranları arasındaki farkı (mutlak fark) hesapla
state_gender_pivot['Fark (Erkek - Kadın)'] = state_gender_pivot['Male'] - state_gender_pivot['Female']

# Farkın en yüksek olduğu eyaletlere göre sırala
state_gender_pivot_sorted = state_gender_pivot.sort_values(by='Fark (Erkek - Kadın)', ascending=False)

print("Erkek ve Kadın Arasındaki Ölüm Oranı Farkının En Yüksek Olduğu 10 Eyalet/Bölge:")
print(state_gender_pivot_sorted[['Male', 'Female', 'Fark (Erkek - Kadın)']].head(10))

print("\nErkek ve Kadın Arasındaki Ölüm Oranı Farkının En Düşük Olduğu 10 Eyalet/Bölge:")
print(state_gender_pivot_sorted[['Male', 'Female', 'Fark (Erkek - Kadın)']].tail(10))
print("-" * 30)


# --- 3. GÖRSELLEŞTİRME: CİNSİYET FARKI BAR GRAFİĞİ ---
print("\nAdım 3: Cinsiyet Farkı Analizi Görselleştiriliyor...")

# Sadece farkın en yüksek olduğu ilk 20 ve son 10'u birleştirelim (grafik çok kalabalık olmasın)
top_20 = state_gender_pivot_sorted.head(20)
bottom_10 = state_gender_pivot_sorted.tail(10)
df_plot = pd.concat([top_20, bottom_10])

# Grafiğin boyutunu ayarla
plt.figure(figsize=(12, 12)) 

# Seaborn barplot kullan
# Not: 'df_plot' zaten sıralı olduğu için 'order' parametresine gerek yok.
sns.barplot(
    x='Fark (Erkek - Kadın)', 
    y=df_plot.index,
    data=df_plot,
    palette="RdBu_r"  # Kırmızılar yüksek, Maviler düşük fark
)

plt.title('Eyaletlere Göre Erkek ve Kadın Ölüm Oranları Arasındaki Fark (Erkek - Kadın)', fontsize=15)
plt.xlabel('Ölüm Oranı Farkı (100.000 Kişi Başına)', fontsize=12)
plt.ylabel('Eyalet / Bölge', fontsize=12)
plt.axvline(x=0, color='black', linestyle='--') # Sıfır çizgisini ekle
plt.tight_layout() # Grafiğin sığmasını sağla

# Grafiği dosyaya kaydet
output_image_file = "cinsiyet_farki_eyalet_bar_chart.png"
plt.savefig(output_image_file, dpi=300)

print(f"Grafik '{output_image_file}' olarak kaydedildi.")
print("\nAdım 1.2 tamamlandı. Kodu çalıştırıp grafiği inceleyebilirsiniz.")

In [ ]:

# --- 2. YENİ ANALİZ: İLÇE (COUNTY) BAZINDA HOTSPOT ANALİZİ ---
print("\nAdım 2: Analiz - İlçe (County) Bazında En Yüksek Riskli Bölgeler (Hotspots)")

# Veriyi filtrele:
# - Sadece 'İlçe' (County) seviyesi
# - Sadece 'Genel' (Overall) Cinsiyet
# - Sadece 'Genel' (Overall) Irk/Etnisite
df_county_overall = df_clean[
    (df_clean['GeographicLevel'] == 'County') &
    (df_clean['Stratification1'] == 'Overall') &
    (df_clean['Stratification2'] == 'Overall')
]

# İlçe bazında ortalama ölüm oranlarını hesapla
# NOT: İlçe adları (LocationDesc) eyaletler arasında tekrarlanabilir (örn. birden fazla "Washington County").
# Bu yüzden hem Eyalet Kısaltması (LocationAbbr) hem de İlçe Adı (LocationDesc) ile grupluyoruz.
# (Veriler 2019-2021 arası ortalama olduğu için zaten genellikle tekil olacaktır, ama bu en güvenli yöntem)
county_mortality = df_county_overall.groupby(
    ['LocationAbbr', 'LocationDesc']
)['Data_Value'].mean().sort_values(ascending=False)

print("--- En Yüksek Ölüm Oranına Sahip İlk 25 İlçe (Hotspots) ---")
print(county_mortality.head(25))

print("\n--- En Düşük Ölüm Oranına Sahip İlk 25 İlçe ---")
print(county_mortality.tail(25))
print("-" * 30)
print("\nAdım 1.3 tamamlandı. İlçe bazlı risk listesini inceleyebilirsiniz.")

In [ ]:
import pandas as pd
import geopandas as gpd  # Artık 'gpd'nin tanınması gerekiyor
import matplotlib.pyplot as plt

# --- 1. VERİ YÜKLEME VE HAZIRLAMA ---
print("Adım 1: Veri Yükleniyor ve Hazırlanıyor...")
file_name = "Heart_Disease_Mortality_Data_Among_US_Adults__35___by_State_Territory_and_County___2019-2021.csv"

try:
    df = pd.read_csv(file_name)
    df_clean = df.dropna(subset=['Data_Value'])
    
    df_county_overall = df_clean[
        (df_clean['GeographicLevel'] == 'County') &
        (df_clean['Stratification1'] == 'Overall') &
        (df_clean['Stratification2'] == 'Overall')
    ]
    
    df_map_data = df_county_overall[['LocationID', 'Data_Value']].copy()
    df_map_data['FIPS'] = df_map_data['LocationID'].astype(str).str.zfill(5)
    
    print(f"Haritalama için {len(df_map_data)} ilçe verisi hazırlandı.")
    print("-" * 30)

except Exception as e:
    print(f"Adım 1'de (Veri Yükleme) bir hata oluştu: {e}")
    # Eğer bu adım başarısız olursa, kodun geri kalanını çalıştırmanın anlamı yok.
    # Bu yüzden buraya bir 'raise' ekleyerek işlemi durduruyoruz.
    raise


# --- ADIM 2, 3 VE 4: HARİTALAMA (BİRLEŞİK BLOK) ---
# Bu adımları birleştirdik, böylece Adım 2 başarısız olursa Adım 3'e geçmez.

try:
    # --- 2. HARİTA VERİSİNİ (SHAPEFILE) YÜKLEME ---
    print("Adım 2: Harita Sınırları (Shapefile) Yükleniyor...")
    
    # --- KULLANICI EYLEMİ 1 ---
    # .shp dosyanızın adını ve yolunu buraya yazın
    shapefile_path = "tl_2023_us_county.shp"  # <-- BU YOLU KONTROL EDİN
    
    # Artık 'gpd'nin tanımlı olması gerekiyor
    gdf_counties = gpd.read_file(shapefile_path)
    print("Harita verisi başarıyla yüklendi.")
    print("-" * 30)

    # --- 3. VERİLERİ BİRLEŞTİRME (Merge) ---
    print("Adım 3: Ölüm Oranı Verisi Harita Verisiyle Birleştiriliyor...")
    
    # --- KULLANICI EYLEMİ 2 ---
    # Harita dosyasındaki FIPS sütununun adı (Genellikle 'GEOID')
    fips_column_in_shapefile = 'GEOID'  # <-- BU SÜTUN ADINI KONTROL EDİN
    
    merged_gdf = gdf_counties.merge(
        df_map_data, 
        left_on=fips_column_in_shapefile,
        right_on='FIPS',
        how='left'
    )
    
    print("Veriler başarıyla birleştirildi.")
    print("-" * 30)

    # --- 4. HARİTAYI OLUŞTURMA VE KAYDETME ---
    print("Adım 4: Harita Oluşturuluyor...")
    
    # Ana kara ABD dışındaki bölgeleri filtrele
    contig_us_map = merged_gdf[
        ~merged_gdf['STATEFP'].isin(['02', '15', '60', '66', '69', '72', '78'])
    ]
    
    fig, ax = plt.subplots(1, 1, figsize=(20, 12))
    
    contig_us_map.plot(
        column='Data_Value',
        ax=ax,
        legend=True,
        cmap='Reds',
        missing_kwds={
            "color": "lightgrey",
            "edgecolor": "white",
            "hatch": "///",
            "label": "Veri Yetersiz/Yok"
        }
    )
    
    ax.set_title('ABD İlçelerinde Kalp Hastalığı Ölüm Oranları (Genel, 2019-2021 Ort.)', fontsize=20)
    ax.set_axis_off()
    
    output_map_file = "ilce_bazli_kalp_hastaligi_haritasi.png"
    plt.savefig(output_map_file, dpi=300, bbox_inches='tight')
    
    print(f"Harita başarıyla '{output_map_file}' olarak kaydedildi.")
    print("\nAdım 2.1 tamamlandı.")

except NameError as ne:
    if 'gpd' in str(ne):
        print("HATA: 'geopandas' (gpd) kütüphanesi tanınmıyor.")
        print("LÜTFEN `pip install geopandas` YAPIN VE KERNEL'İ YENİDEN BAŞLATIN.")
    else:
        print(f"Bir İsim Hatası (NameError) oluştu: {ne}")

except FileNotFoundError as fnf:
    print(f"HATA: Dosya bulunamadı: {fnf.filename}")
    print(f"Lütfen '{shapefile_path}' dosyasının doğru klasörde olduğundan emin olun.")

except Exception as e:
    print(f"Haritalama sırasında beklenmedik bir hata oluştu: {e}")